In [ ]:
library(tidyverse)
library(tidymodels)
library(repr)
library(cowplot)
options(repr.matrix.max.rows = 6)
source('cleanup.R')

In [ ]:
set.seed(1000)

# Optimal K

players_short <- players_short |>
    mutate(subscribe = as.factor(subscribe))

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
    set_engine("kknn") |>
    set_mode("classification")

data_split <- initial_split(players_short, prop = 0.7, strata = subscribe) 
    train_data <- training(data_split)
    test_data  <- testing(data_split)

knn_recipe <- recipe(subscribe ~ Age, data = train_data) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors())

knn_wf <- workflow() |>
    add_model(knn_spec) |>
    add_recipe(knn_recipe)

k_grid <- grid_regular(neighbors(range = c(1, 10)), levels = 10)

k_vfold <- vfold_cv(train_data, v = 5)

knn_tuned <- tune_grid(knn_wf, resamples = k_vfold, grid = k_grid, metrics = metric_set(accuracy))

k_select <- select_best(knn_tuned, metric = "accuracy")
k_select

In [ ]:
sub_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = k_select) |>
    set_engine("kknn") |>
    set_mode("classification")

sub_recipe <- recipe(subscribe ~ Age, data = players_short) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors())

sub_workflow <- workflow() |>
    add_model(sub_spec) |>
    add_recipe(sub_recipe)

sub_workflow